# [CDAF] Atividade 2

## Nome: Thiago Pádua de Carvalho

## Matrícula: 2020007066

## Introdução
Nesta atividade, vamos revisar os conceitos aprendidos em sala de aula sobre estatísticas agregadas. Para esta atividade, usaremos dados do Brasileirão 2022 do FBRef.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

## Questão 1
- Baixe o dataset de resultados em https://fbref.com/en/comps/24/2022/schedule/2022-Serie-A-Scores-and-Fixtures
- Crie uma média móvel de 5 jogos, para cada equipe, de cada uma das seguintes estatísticas: xG pró, xG contra, e dif. xG.
- Escolha 4 times para visualizar a série temporal das estatísticas acima. Uma visualização para cada uma das estatísticas, onde a média geral do campeonato é apresentada com uma linha pontilhada em conjunto com a média móvel dos times escolhidos.
- Interprete os resultados. O que isso pode indicar sobre a qualidade ofensiva e defensiva dos times escolhidos?

In [2]:
BRA1 = pd.read_csv('data/brazilian_league_20222.csv', sep=',')
BRA1.drop(columns=['Match Report', 'Notes', 'Attendance', 'Venue', 'Time', 'Referee', 'Day'], inplace=True)
BRA1 = BRA1.dropna()
display(BRA1)

,Wk,Date,Home,xG,Score,xG.1,Away
0,1.0,2022-04-09,Fluminense,1.6,0–0,0.1,Santos
1,1.0,2022-04-09,Atl Goianiense,1.4,1–1,1.2,Flamengo
2,1.0,2022-04-10,Coritiba,1.7,3–0,0.4,Goiás
3,1.0,2022-04-10,Atlético Mineiro,1.9,2–0,0.6,Internacional
4,1.0,2022-04-10,Botafogo (RJ),1.5,1–3,2.1,Corinthians
...,...,...,...,...,...,...,...
417,38.0,2022-11-13,Bragantino,0.9,0–1,0.6,Fluminense
418,38.0,2022-11-13,América (MG),2.1,1–1,0.5,Atl Goianiense
419,38.0,2022-11-13,Goiás,0.2,0–4,1.7,São Paulo
420,38.0,2022-11-13,Internacional,1.4,3–0,1.3,Palmeiras


In [3]:
def split_score(score):
    """
    splits match's score in format 'x-x' into two integer numbers.
    
    Args:
    score (str): Score in format 'x-x', where x is an integer number.
    
    Returns:
    Two integers: home score and away score
    """
    split_score = score.split("-")
    HS = int(split_score[0])
    AS = int(split_score[1])
    return HS, AS

In [4]:
def assign_xg(match, home=True):
    if home:
        xg_pro = match['xG']
        xg_con = match['xG.1']
    else:
        xg_pro = match['xG.1']
        xg_con = match['xG']
    return xg_pro, xg_con, xg_pro - xg_con

In [7]:
def rolling_average(teams, window=5):
    teams_dataframe = {}
    for team in teams:
        team_df = pd.DataFrame(columns=['xg_pro', 'xg_con', 'xg_diff', 'Game'])
        matches = BRA1.loc[(BRA1['Home'] == team) | (BRA1['Away'] == team)]
        game = 0
        for i, match in matches.iterrows():
            game += 1
            if match['Home'] == team:
                xg_pro, xg_con, xg_diff = assign_xg(match)
            else:
                xg_pro, xg_con, xg_diff = assign_xg(match, home=False)
            
            team_df.at[i, "xg_pro"] = xg_pro
            team_df.at[i, "xg_con"] = xg_con
            team_df.at[i, "xg_diff"] = xg_diff
            team_df.at[i, "Game"] = game

        team_df['XG_RA'] = team_df['xg_pro'].rolling(window=window, win_type='triang').mean()
        team_df['XG_con_RA'] = team_df['xg_con'].rolling(window=window, win_type='triang').mean()
        team_df['XG_RA'] = team_df['xg_diff'].rolling(window=window, win_type='triang').mean()
        team_df['XG_RA'] = team_df['xg_pro'].rolling(window=window, win_type='triang').mean()

        teams_dataframe[team] = team_df
    
    return teams_dataframe

In [9]:
teams = list(BRA1['Home'].unique())
teams_df = rolling_average(teams)
teams_df

{'Fluminense':     xg_pro xg_con xg_diff Game     XG_RA  XG_con_RA
 0      1.6    0.1     1.5    1       NaN        NaN
 20     0.7    0.2     0.5    2       NaN        NaN
 26     0.5    0.8    -0.3    3       NaN        NaN
 40     0.6    3.3    -2.7    4       NaN        NaN
 48     0.7    2.1    -1.4    5  0.711111   1.288889
 66     0.7    1.4    -0.7    6  0.622222   1.922222
 73     1.1    1.7    -0.6    7  0.700000   2.022222
 84     1.6    1.0     0.6    8  0.877778   1.788889
 96     0.3    1.1    -0.8    9  0.988889   1.455556
 111    1.8    0.8     1.0   10  1.122222   1.200000
 116    1.8    1.9    -0.1   11  1.177778   1.166667
 133    1.3    1.0     0.3   12  1.388889   1.155556
 142    1.7    0.5     1.2   13  1.511111   1.211111
 151    0.8    0.7     0.1   14  1.500000   1.033333
 157    2.3    0.3     2.0   15  1.488889   0.788889
 169    1.5    1.8    -0.3   16  1.466667   0.722222
 183    1.2    0.8     0.4   17  1.600000   0.800000
 191    1.1    1.0     0.1   18 

https://soccermatics.readthedocs.io/en/latest/gallery/lesson5/plot_PointsMovingAverage.html#points-moving-average

## Questão 2
- Agora repita a questão 1, plotando as séries temporais dos mesmos times, mas para uma janela móvel de 10 jogos.
- Quais as diferenças entre as séries temporais de 5 e 10 jogos? Em quais situações pode ser vantajoso escolher uma janela à outra?

## Questão 3
- Vá para o link
-- https://fbref.com/en/comps/24/2022/stats/2022-Serie-A-Stats
- Nesta seção de estatísticas, é possível navegar por estatísticas específicas para diferentes aspectos do jogo (finalização, passe, defesa, etc.). Para todos exercícios à partir deste, você terá que selecionar aquelas que julgar mais relevantes para responder as questões.
- Monte um radar plot com 6 atributos relevantes para atacantes e compare 3 jogadores de sua escolha. Justifique a escolha de cada um dos atributos, a escolha da escala dos radares e o tipo de normalização. Interprete os resultados dos radares em termos das qualidades e limitações dos jogadores.
- Ref Soccermatics:
-- https://soccermatics.readthedocs.io/en/latest/lesson3/ScoutingPlayers.html
-- https://soccermatics.readthedocs.io/en/latest/gallery/lesson3/plot_RadarPlot.html

## Questão 4
- Faça o mesmo que na questão 3, mas para meio campistas.

## Questão 5
- Faça o mesmo que na questão 3, mas para zagueiros.

## Questão 6
- Discuta as diferenças entre os radares das questões 3, 4 e 5. Quais são as principais diferenças entre os atributos relevantes para cada posição? Quais são as principais semelhanças? A impressão subjetiva que você tinha dos jogadores se comprovou pelos radares? Se não, por quê? Quais posições são mais difíceis de serem avaliadas por estatísticas?